In [ ]:
# версии вебуи
newest_commit =  "394ffa7b0a7fff3ec484bcd084e673a8b301ccc8"
previous_commit = "89f9faa63388756314e8a1d96cf86bf5e0663045"
legacy_commit = "b08500cec8a791ef20082628b49b17df833f5dda"
obsolete_commit = "72cd27a13587c9579942577e9e3880778be195f6"
# элементы виджета
checkpoint = widgets.Dropdown(options=[('v1.4.0', 1), ('v1.2.1', 2), ('v1.2.0', 3), ('v1.1.0', 4)], value=1, description='версия A1111:').add_class("checkpoint")
gdrive = widgets.Checkbox(value=False, description='подключить гуглодиск').add_class("googledisk_mount")
gdrive_models = widgets.Checkbox(value=False, description='подтянуть модели/лоры/etc с гуглодиска').add_class("googledisk_symlink")
gfolder = widgets.Text(value="/content/drive/MyDrive/SD/", placeholder='там где лежат папки с моделями и т.п.').add_class("gfolder")
gdrive_only = widgets.Checkbox(value=False, description='модели только с гуглодиска').add_class("googledisk_only")
opt = widgets.RadioButtons(options=['ничего', 'SDP', 'ToMe', 'xformers'], value='SDP', description='Оптимизация:').add_class("opt")
pix2pix = widgets.Checkbox(value=False, description='instruct pix2pix').add_class("pix2pix")
controlnet = widgets.Checkbox(value=False, description='ControlNet').add_class("controlnet")
extentions = widgets.Checkbox(value=False, description='дополнения').add_class("extentions")
extentions_mix = widgets.Checkbox(value=False, description='дополнения для слияние моделей').add_class("extentions_mix")
booru = widgets.Checkbox(value=False, description='DeepDanbooru').add_class("booru")
no_half_vae = widgets.Checkbox(value=True, description='--no-half-vae').add_class("no_half_vae")
lycoris_locon = widgets.Dropdown(options=[('locon', 1), ('lycoris', 2)], value=2, description='дополнение для lycoris:').add_class("lycoris_locon")
starton = widgets.Checkbox(value=True, description='запустить сразу').add_class("starton")
gd_set = widgets.HBox([gdrive, gdrive_models, gfolder, gdrive_only]).add_class("gd_set")
eng = widgets.Checkbox(value=False, description='англоязычный интерфейс').add_class("eng")
send_arts = widgets.Checkbox(value=False, description='поделиться генерациями').add_class("send_arts")
ngrok_token = widgets.Text(value="", placeholder='тут можно указать свой токен ngrok').add_class("ngrok_token")
auth_token = widgets.HTML(value='''<a class="auth_token" title="получить свой токен Ngrok" href="https://dashboard.ngrok.com/get-started/your-authtoken" target="_blank">?</a>''')
# необязательный контент
additional_options = widgets.Accordion().add_class("additional_options")
additional_options.set_title(0, 'дополнительные опции')
additional_options.selected_index = None
loras = widgets.Checkbox(value=False, description='Gachi-LoRa').add_class("loras")
embeddings = widgets.Checkbox(value=False, description='внедрения').add_class("embeddings")
hypernet = widgets.Checkbox(value=False, description='гиперсети').add_class("hypernet")
additional_options.children = [widgets.HBox([loras, embeddings, hypernet]).add_class("extra_options")]
additions = widgets.HBox([controlnet, pix2pix, extentions, extentions_mix, booru, additional_options]).add_class("additions")
webuiargs = widgets.HBox([no_half_vae, lycoris_locon, starton, eng, ngrok_token, auth_token, send_arts]).add_class("args")
apply = widgets.Button(description='продолжить ❯').add_class("apply")
model_header = HTML('<div><h2 id="model_header">выбор моделей</h2><p id="model_description"><a href="https://pr0lapse.github.io/models.html" target="_blank">описания моделей, ключи для их вызова</p></div>')
settings_first_header = HTML('<h2 id="setup_header">настройки</h2>')

#показать/спрятать указание папки на гдиске с моделями
def gd_path(change):
  if gdrive_models.value == True:
    gfolder.layout.display = 'flex'
    gdrive_only.layout.display = 'flex'
  if gdrive_models.value == False:
    gfolder.layout.display = 'none'
    gdrive_only.layout.display = 'none'

# примонтирование диска и показать/спрятать опции гдиска
def gdrive_mount(change):
  if gdrive.value == True:
    gdrive_models.layout.display = 'flex'
    gdrive_only.layout.display = 'flex'
    if not os.path.exists("/content/drive/MyDrive/"):
      try:
        drive.mount('/content/drive', force_remount=True)
        print('гуглодиск примонтирован')
      except:
        print("пользователь отказался")
  if gdrive.value == False:
    print("гуглодиск не примонтирован")
    gdrive_models.layout.display = 'none'
    gdrive_only.layout.display = 'none'
    gfolder.layout.display = 'none'
    gdrive_models.value = False
    gfolder.value = ""
    gdrive_only.value = False

#показать/спрятать виджет моделей (если модели только с гдиска - виджет не нужен)
def only_gdrive(b):
  if gdrive_only.value == True:
    tab.layout.display = 'none'
    search_and_button.layout.display = 'none'
    display(HTML('<style>#model_header,#model_description{display:none}'))
  if gdrive_only.value == False:
    tab.layout.display = 'flex'
    search_and_button.layout.display = 'flex'
    display(HTML('<style>#model_header,#model_description{display:block}'))

# установка вебуи и основных зависимостей, правки
# функция правок в коде
def sed(file_path, line_to_replace, new_line):
  try:
    with open(file_path, 'r') as file: lines = file.readlines()
    with open(file_path, 'w') as file:
      for line in lines:
          if line_to_replace in line:
            line = line.replace(line_to_replace, new_line)
          file.write(line)
  except: pass
# функция удаления и замены строк в коде и текстах
def replacer(file_path, line_to_remove, new):
  try:
    with open(file_path) as r:
      text = r.read().replace(f"""{line_to_remove}""", f"""{new}""")
    with open(file_path, "w") as w: w.write(text)
  except: pass
# функция установки
def A1111_webui_install():
  try:
    subprocess.run(["git", "clone", "https://github.com/AUTOMATIC1111/stable-diffusion-webui"])
    os.chdir('/content/stable-diffusion-webui')
    if checkpoint_value == "newest":
      subprocess.run(["git", "checkout", newest_commit], cwd="/content/stable-diffusion-webui")
    if checkpoint_value == "previous":
      subprocess.run(["git", "checkout", previous_commit], cwd="/content/stable-diffusion-webui")
    if checkpoint_value == "legacy":
      subprocess.run(["git", "checkout", legacy_commit], cwd="/content/stable-diffusion-webui")
    if checkpoint_value == "obsolete":
      subprocess.run(["git", "checkout", obsolete_commit], cwd="/content/stable-diffusion-webui")
    # замена строк из файла зависимостей, для предотвращения переустановки
    replacer("/content/stable-diffusion-webui/requirements_versions.txt", "gradio==3.32.0", "gradio==3.33.0")
    replacer("/content/stable-diffusion-webui/requirements.txt", "gradio==3.32.0", "gradio==3.33.0")
    # установка репозиториев для вебуи
    if checkpoint_value == "newest":
      launch = "modules/launch_utils.py"
    else:
      launch = "launch.py"
    try:
      openclip = "/root/.cache/huggingface/hub/models--openai--clip-vit-large-patch14/snapshots/8d052a0f05efbaefbc9e8786ba291cfdf93e5bff/"
      os.makedirs(openclip, exist_ok=True)
      with open(f'{launch}', 'r') as fp:
        lines = fp.readlines()
        for line in lines:
          if line.find('STABLE_DIFFUSION_COMMIT_HASH') != -1:
            stable_diffusion_commit_hash = line.split('"')[1]
          if checkpoint_value != "newest":
            if line.find('TAMING_TRANSFORMERS_COMMIT_HASH') != -1:
              taming_transformers_commit_hash = line.split('"')[1]
          if line.find('K_DIFFUSION_COMMIT_HASH') != -1:
            k_diffusion_commit_hash = line.split('"')[1]
          if line.find('CODEFORMER_COMMIT_HASH') != -1:
            codeformer_commit_hash = line.split('"')[1]
          if line.find('BLIP_COMMIT_HASH') != -1:
            blip_commit_hash = line.split('"')[1]
      if checkpoint_value != "newest":
        taming_transformers = f"git clone https://github.com/CompVis/taming-transformers.git /content/stable-diffusion-webui/repositories/taming-transformers && cd /content/stable-diffusion-webui/repositories/taming-transformers && git checkout {taming_transformers_commit_hash}"
      else:
        taming_transformers = ""
      reps = [
          f"git clone https://github.com/Stability-AI/stablediffusion.git /content/stable-diffusion-webui/repositories/stable-diffusion-stability-ai && cd /content/stable-diffusion-webui/repositories/stable-diffusion-stability-ai && git checkout {stable_diffusion_commit_hash}",
          f"{taming_transformers}",
          f"git clone https://github.com/crowsonkb/k-diffusion.git /content/stable-diffusion-webui/repositories/k-diffusion && cd /content/stable-diffusion-webui/repositories/k-diffusion && git checkout {k_diffusion_commit_hash}",
          f"git clone https://github.com/sczhou/CodeFormer.git /content/stable-diffusion-webui/repositories/CodeFormer && cd /content/stable-diffusion-webui/repositories/CodeFormer && git checkout {codeformer_commit_hash}",
          f"git clone https://github.com/salesforce/BLIP.git /content/stable-diffusion-webui/repositories/BLIP && cd /content/stable-diffusion-webui/repositories/BLIP && git checkout {blip_commit_hash}",
          f"git clone https://github.com/isl-org/MiDaS.git /content/stable-diffusion-webui/repositories/midas",
          "wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O /usr/bin/cloudflared  && chmod +x /usr/bin/cloudflared",
          f"wget https://huggingface.co/utnah/clip/resolve/main/pytorch_model.bin -P {openclip}",
          ]
      def reps_inst(rep):
        !$rep
      with ThreadPoolExecutor(max_workers=len(reps)) as executor:
        futures = [executor.submit(reps_inst, rep) for rep in reps]
        for future in as_completed(futures):
            future.result()
    except:
      None
    !$wget {deps}cache.tar
    !tar -xvf cache.tar -C /
    !rm cache.tar
    os.environ["SAFETENSORS_FAST_GPU"]='1'
    if (checkpoint_value != "newest") and (checkpoint_value != "previous"):
      print("\033[1;32mподождите, происходит переустановка зависимостей для старой версии, это не быстро...")
      !pip uninstall -y torchaudio torchvision torchtext xformers triton  > /dev/null 2>&1
      !python launch.py --share --xformers --ngrok test --no-download-sd-model --exit
    os.chdir('/content/stable-diffusion-webui')
    # оптимизации
    sed("/content/stable-diffusion-webui/webui.py", "ui.create_ui()", "ui.create_ui();shared.demo.queue(concurrency_count=999999,status_update_rate=0.1)")
    sed("/content/stable-diffusion-webui/modules/sd_models.py", "os.path.splitext(checkpoint_file)", "os.path.splitext(checkpoint_file); map_location='cuda'")
    sed("/content/stable-diffusion-webui/modules/extras.py", "map_location='cpu'", "map_location='cuda'")
    sed("/content/stable-diffusion-webui/modules/shared.py", 'cmd_opts.lowram else "cpu"', 'cmd_opts.lowram else "cuda"')
    replacer("/content/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/util.py", 'return get_obj_from_str(config["target"])(**config.get("params", dict()))', 'return get_obj_from_str(config["target"])(**config.get("params", dict())).cuda()')
    for root, dirs, files in os.walk("/content/stable-diffusion-webui/"):
      for file in files:
        if file.endswith('here.txt'):
          os.remove(os.path.join(root, file))
    clear()
  except Exception as e:
    try:
      error_message = str(cgitb.text(sys.exc_info()))
      e = "в функции A1111_webui_install и получил "+str(e)
      ErrorReport(e, error_message)
      print("Ошибка: ", e)
    except Exception as e:
      import traceback
      error_message = str(traceback.format_exc())
      e = "в функции A1111_webui_install и получил "+str(e)
      ErrorReport(e, error_message)
      print("Ошибка: ", error_message)

# скачивание модели и дополнения instruct-pix2pix
def instruct_pix2pix():
  try:
    if pix2pix.value == True:
      instructs = [
          "git clone https://github.com/Klace/stable-diffusion-webui-instruct-pix2pix /content/stable-diffusion-webui/extensions/stable-diffusion-webui-instruct-pix2pix",
          f"{dl} {sf}instruct-pix2pix.safetensors -c -P /content/stable-diffusion-webui/models/Stable-diffusion/",
        ]
      def pix2pix_ext(instruct):
          !$instruct
      with ThreadPoolExecutor(max_workers=len(instructs)) as executor:
          futures = [executor.submit(pix2pix_ext, instruct) for instruct in instructs]
          for future in as_completed(futures):
              future.result()
  except Exception as e:
    try:
      error_message = str(cgitb.text(sys.exc_info()))
      e = "в функции instruct_pix2pix и получил "+str(e)
      ErrorReport(e, error_message)
      print("Ошибка: ", e)
    except Exception as e:
      import traceback
      e = "в функции instruct_pix2pix и получил "+str(e)
      error_message = str(traceback.format_exc())
      ErrorReport(e, error_message)
      print("Ошибка: ", error_message)
  if pix2pix.value == False:
    None
  clear()

# скачивание моделей и дополнения ControlNet
def ControlNet_install():
  try:
    if controlnet.value == True:
      subprocess.run(["git", "clone", "--depth", "1", "https://github.com/Mikubill/sd-webui-controlnet"], cwd="/content/stable-diffusion-webui/extensions")
      replacer("/content/stable-diffusion-webui/extensions/sd-webui-controlnet/scripts/controlnet.py", '# "openpose_hand": openpose_hand,', '"openpose_hand": openpose_hand,')
      os.remove(f'/content/stable-diffusion-webui/extensions/sd-webui-controlnet/models/cldm_v15.yaml')
      dlcn = f"wget -nv -t 10 --show-progress --progress=bar -q --content-disposition -P /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models "
      cnrepo = "https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/"
      cnurl = sf+"controlnet/"
      controlnets = [
          "pip install svglib > /dev/null 2>&1",
          "pip install mediapipe > /dev/null 2>&1",
          "pip install fvcore > /dev/null 2>&1",
          f"{dlcn} https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/cldm_v15.yaml",
          f"{dlcn} {cnurl}control_hed.safetensors",
          f"{dlcn} {cnurl}control_normal.safetensors",
          f"{dlcn} {cnrepo}control_v11e_sd15_ip2p_fp16.safetensors",
          f"{dlcn} {cnrepo}control_v11e_sd15_shuffle_fp16.safetensors",
          f"{dlcn} {cnrepo}control_v11p_sd15_canny_fp16.safetensors",
          f"{dlcn} {cnrepo}control_v11f1p_sd15_depth_fp16.safetensors",
          f"{dlcn} {cnrepo}control_v11p_sd15_inpaint_fp16.safetensors",
          f"{dlcn} {cnrepo}control_v11p_sd15_lineart_fp16.safetensors",
          f"{dlcn} {cnrepo}control_v11p_sd15_mlsd_fp16.safetensors",
          f"{dlcn} {cnrepo}control_v11p_sd15_normalbae_fp16.safetensors",
          f"{dlcn} {cnrepo}control_v11p_sd15_openpose_fp16.safetensors",
          f"{dlcn} {cnrepo}control_v11p_sd15_scribble_fp16.safetensors",
          f"{dlcn} {cnrepo}control_v11p_sd15_seg_fp16.safetensors",
          f"{dlcn} {cnrepo}control_v11p_sd15_softedge_fp16.safetensors",
          f"{dlcn} {cnrepo}control_v11p_sd15s2_lineart_anime_fp16.safetensors",
          f"{dlcn} {cnrepo}control_v11f1e_sd15_tile_fp16.safetensors",
          f"{dlcn} {cnrepo}control_v11e_sd15_ip2p_fp16.yaml",
          f"{dlcn} {cnrepo}control_v11e_sd15_shuffle_fp16.yaml",
          f"{dlcn} {cnrepo}control_v11p_sd15_canny_fp16.yaml",
          f"{dlcn} {cnrepo}control_v11f1p_sd15_depth_fp16.yaml",
          f"{dlcn} {cnrepo}control_v11p_sd15_inpaint_fp16.yaml",
          f"{dlcn} {cnrepo}control_v11p_sd15_lineart_fp16.yaml",
          f"{dlcn} {cnrepo}control_v11p_sd15_mlsd_fp16.yaml",
          f"{dlcn} {cnrepo}control_v11p_sd15_normalbae_fp16.yaml",
          f"{dlcn} {cnrepo}control_v11p_sd15_openpose_fp16.yaml",
          f"{dlcn} {cnrepo}control_v11p_sd15_scribble_fp16.yaml",
          f"{dlcn} {cnrepo}control_v11p_sd15_seg_fp16.yaml",
          f"{dlcn} {cnrepo}control_v11p_sd15_softedge_fp16.yaml",
          f"{dlcn} {cnrepo}control_v11p_sd15s2_lineart_anime_fp16.yaml",
          f"{dlcn} {cnrepo}control_v11f1e_sd15_tile_fp16.yaml",
          f"{dlcn} {cnrepo}t2iadapter_style_sd14v1.pth",
          f"{dlcn} {cnrepo}t2iadapter_sketch_sd14v1.pth",
          f"{dlcn} {cnrepo}t2iadapter_seg_sd14v1.pth",
          f"{dlcn} {cnrepo}t2iadapter_openpose_sd14v1.pth",
          f"{dlcn} {cnrepo}t2iadapter_keypose_sd14v1.pth",
          f"{dlcn} {cnrepo}t2iadapter_depth_sd14v1.pth",
          f"{dlcn} {cnrepo}t2iadapter_color_sd14v1.pth",
          f"{dlcn} {cnrepo}t2iadapter_canny_sd14v1.pth",
          f"{dlcn} {cnrepo}t2iadapter_canny_sd15v2.pth",
          f"{dlcn} {cnrepo}t2iadapter_depth_sd15v2.pth",
          f"{dlcn} {cnrepo}t2iadapter_sketch_sd15v2.pth",
          f"{dlcn} {cnrepo}t2iadapter_zoedepth_sd15v1.pth",
          f"{wget} https://huggingface.co/lllyasviel/ControlNet/resolve/main/annotator/ckpts/dpt_hybrid-midas-501f0c75.pt -P /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/midas",
          f"{wget} https://huggingface.co/lllyasviel/ControlNet/resolve/main/annotator/ckpts/mlsd_large_512_fp32.pth -P /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/mlsd",
          f"{wget} https://huggingface.co/lllyasviel/ControlNet/resolve/main/annotator/ckpts/mlsd_tiny_512_fp32.pth -P /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/mlsd",
          f"{wget} https://huggingface.co/lllyasviel/ControlNet/resolve/main/annotator/ckpts/upernet_global_small.pth -P /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/uniformer",
          f"{wget} https://huggingface.co/lllyasviel/ControlNet/resolve/main/annotator/ckpts/hand_pose_model.pth -P /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/openpose",
          f"{wget} https://huggingface.co/lllyasviel/ControlNet/resolve/main/annotator/ckpts/body_pose_model.pth -P /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/openpose",
          f"{wget} https://huggingface.co/lllyasviel/ControlNet/resolve/main/annotator/ckpts/network-bsds500.pth -P /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/hed",
          f"git clone https://github.com/fkunn1326/openpose-editor /content/stable-diffusion-webui/extensions/openpose-editor",
          f"git clone https://github.com/hnmr293/posex /content/stable-diffusion-webui/extensions/posex",
          f"git clone https://github.com/nonnonstop/sd-webui-3d-open-pose-editor /content/stable-diffusion-webui/extensions/3d-open-pose-editor",
          f"git clone https://github.com/jexom/sd-webui-depth-lib /content/stable-diffusion-webui/extensions/depth-lib",
        ]
      def controlnet_ext(controlnet):
          !$controlnet
      with ThreadPoolExecutor(max_workers=len(controlnets)) as executor:
          futures = [executor.submit(controlnet_ext, controlnet) for controlnet in controlnets]
          for future in as_completed(futures):
              future.result()
      import shutil
      controlnet_models_path = "/content/stable-diffusion-webui/extensions/sd-webui-controlnet/models/"
      cn_cofigs = ["control_sd15_canny.yaml", "control_sd15_depth.yaml", "control_sd15_hed.yaml", "control_sd15_mlsd.yaml", "control_sd15_normal.yaml", "control_sd15_openpose.yaml", "control_sd15_scribble.yaml", "control_sd15_seg.yaml"]
      for cn_cofig in cn_cofigs:
        shutil.copyfile(os.path.join(controlnet_models_path, cn_cofig), os.path.join(controlnet_models_path, cn_cofig.replace("_sd15", "")))
      dmaps = "/content/stable-diffusion-webui/extensions/3d-open-pose-editor/maps/"
      !rm -rf {dmaps}*
      subprocess.run(["wget", "-t", "10", "--content-disposition", "-O", "maps.7z", "https://raw.githubusercontent.com/PR0LAPSE/StableDiffusionWebUIColab/main/src/maps.7z"], cwd="/content/")
      !7z -bso0 -bd -mmt4 -slp -y x /content/maps.7z -o{dmaps}
      !rm /content/maps.7z
    clear()
  except Exception as e:
    try:
      error_message = str(cgitb.text(sys.exc_info()))
      e = "в функции ControlNet_install и получил "+str(e)
      ErrorReport(e, error_message)
      print("Ошибка: ", e)
    except Exception as e:
      import traceback
      error_message = str(traceback.format_exc())
      e = "в функции ControlNet_install и получил "+str(e)
      ErrorReport(e, error_message)
      print("Ошибка: ", error_message)
  if controlnet.value == False:
    None

# дополнение clip-interrogator
def interrogate_clip_install():
  try:
    subprocess.run(["git", "clone", "--depth", "1", "https://github.com/pharmapsychotic/clip-interrogator-ext"], cwd="/content/stable-diffusion-webui/extensions")
    ipath = "/content/stable-diffusion-webui/models/clip-interrogator"
    dl = f"{wget} -P {ipath} "
    clips = [
        f"{dl} https://huggingface.co/pharma/ci-preprocess/resolve/main/RN50_openai_artists.safetensors",
        f"{dl} https://huggingface.co/pharma/ci-preprocess/resolve/main/RN50_openai_flavors.safetensors",
        f"{dl} https://huggingface.co/pharma/ci-preprocess/resolve/main/RN50_openai_mediums.safetensors",
        f"{dl} https://huggingface.co/pharma/ci-preprocess/resolve/main/RN50_openai_movements.safetensors",
        f"{dl} https://huggingface.co/pharma/ci-preprocess/resolve/main/RN50_openai_negative.safetensors",
        f"{dl} https://huggingface.co/pharma/ci-preprocess/resolve/main/RN50_openai_trendings.safetensors",
        f"{dl} https://huggingface.co/pharma/ci-preprocess/resolve/main/ViT-H-14_laion2b_s32b_b79k_artists.safetensors",
        f"{dl} https://huggingface.co/pharma/ci-preprocess/resolve/main/ViT-H-14_laion2b_s32b_b79k_flavors.safetensors",
        f"{dl} https://huggingface.co/pharma/ci-preprocess/resolve/main/ViT-H-14_laion2b_s32b_b79k_mediums.safetensors",
        f"{dl} https://huggingface.co/pharma/ci-preprocess/resolve/main/ViT-H-14_laion2b_s32b_b79k_movements.safetensors",
        f"{dl} https://huggingface.co/pharma/ci-preprocess/resolve/main/ViT-H-14_laion2b_s32b_b79k_negative.safetensors",
        f"{dl} https://huggingface.co/pharma/ci-preprocess/resolve/main/ViT-H-14_laion2b_s32b_b79k_trendings.safetensors",
        f"{dl} https://huggingface.co/pharma/ci-preprocess/resolve/main/ViT-L-14_openai_artists.safetensors",
        f"{dl} https://huggingface.co/pharma/ci-preprocess/resolve/main/ViT-L-14_openai_flavors.safetensors",
        f"{dl} https://huggingface.co/pharma/ci-preprocess/resolve/main/ViT-L-14_openai_mediums.safetensors",
        f"{dl} https://huggingface.co/pharma/ci-preprocess/resolve/main/ViT-L-14_openai_movements.safetensors",
        f"{dl} https://huggingface.co/pharma/ci-preprocess/resolve/main/ViT-L-14_openai_negative.safetensors",
        f"{dl} https://huggingface.co/pharma/ci-preprocess/resolve/main/ViT-L-14_openai_trendings.safetensors",
        "pip install clip-interrogator==0.5.4 > /dev/null 2>&1",
        ]
    def clips_inst(clip):
      !$clip
    with ThreadPoolExecutor(max_workers=len(clips)) as executor:
      futures = [executor.submit(clips_inst, clip) for clip in clips]
      for future in as_completed(futures):
          future.result()
      clear()
  except Exception as e:
    try:
      error_message = str(cgitb.text(sys.exc_info()))
      e = "в функции interrogate_clip_install и получил "+str(e)
      ErrorReport(e, error_message)
      print("Ошибка: ", e)
    except Exception as e:
      import traceback
      e = "в функции interrogate_clip_install и получил "+str(e)
      error_message = str(traceback.format_exc())
      ErrorReport(e, error_message)
      print("Ошибка: ", error_message)

# обязательные дополнения
#f"git clone https://github.com/wcde/custom-hires-fix-for-automatic1111 /content/stable-diffusion-webui/extensions/custom-hires-fix",
#f"git clone https://github.com/hnmr293/sd-webui-llul /content/stable-diffusion-webui/extensions/sd-webui-llul",
#f"git clone https://github.com/KohakuBlueleaf/a1111-sd-webui-locon /content/stable-diffusion-webui/extensions/a1111-sd-webui-locon",
def extentions_builtin():
  try:
    if lycoris_locon_value == "locon":
      lyco_ext = f"git clone https://github.com/KohakuBlueleaf/a1111-sd-webui-locon /content/stable-diffusion-webui/extensions/a1111-sd-webui-locon"
    if lycoris_locon_value == "lycoris":
      lyco_ext = f"git clone https://github.com/KohakuBlueleaf/a1111-sd-webui-lycoris /content/stable-diffusion-webui/extensions/a1111-sd-webui-lycoris"
    #таймер, описания моделей и апскейлер, замена лиц, пропорции картинки, генератор промптов
    os.makedirs('/content/stable-diffusion-webui/static', exist_ok = True)
    if checkpoint_value != "newest":
      wuifix =  f"git clone -b old https://github.com/PR0LAPSE/webui-fix.git /content/stable-diffusion-webui/extensions-builtin/fix-webui"
    else:
      wuifix =  f"git clone https://github.com/PR0LAPSE/webui-fix /content/stable-diffusion-webui/extensions-builtin/fix-webui"
    builtins = [
      f"{wuifix}",
      f"git clone https://github.com/PR0LAPSE/photopea-a1111 /content/stable-diffusion-webui/extensions/photopea-a1111",
      f"git clone https://github.com/PR0LAPSE/colab_models_list /content/stable-diffusion-webui/extensions-builtin/colab_models_list",
      f"git clone https://github.com/richrobber2/canvas-zoom /content/stable-diffusion-webui/extensions/canvas-zoom",
      f"git clone https://github.com/Coyote-A/ultimate-upscale-for-automatic1111 /content/stable-diffusion-webui/extensions/ultimate-upscale-for-automatic1111",
      f"git clone https://github.com/kohya-ss/sd-webui-additional-networks /content/stable-diffusion-webui/extensions/sd-webui-additional-networks",
      f"{lyco_ext}",
      f"git clone https://github.com/adieyal/sd-dynamic-prompts /content/stable-diffusion-webui/extensions/sd-dynamic-prompts",
      f"git clone https://github.com/yownas/shift-attention /content/stable-diffusion-webui/extensions/shift-attention",
      f"git clone https://github.com/yownas/seed_travel /content/stable-diffusion-webui/extensions/seed_travel",
      f"git clone https://github.com/Filarius/video2video /content/stable-diffusion-webui/extensions/video2video",
      f"git clone https://github.com/kex0/batch-face-swap /content/stable-diffusion-webui/extensions/batch-face-swap",
      f"git clone https://github.com/pkuliyi2015/sd-webui-stablesr  /content/stable-diffusion-webui/extensions/sd-webui-stablesr",
      "pip install mediapipe dynamicprompts > /dev/null 2>&1",
      f"git clone https://github.com/thomasasfk/sd-webui-aspect-ratio-helper /content/stable-diffusion-webui/extensions/sd-webui-aspect-ratio-helper",
      f"git clone https://github.com/Bing-su/adetailer.git /content/stable-diffusion-webui/extensions/adetailer",
    ]
    def ext_builtin_inst(builtin):
      !$builtin
    with ThreadPoolExecutor(max_workers=len(builtins)) as executor:
      futures = [executor.submit(ext_builtin_inst, builtin) for builtin in builtins]
      for future in as_completed(futures):
          future.result()
    !echo -n {launching} > /content/stable-diffusion-webui/static/launch.txt
    if lycoris_locon_value == "locon":
      locon_path = "/content/stable-diffusion-webui/extensions/a1111-sd-webui-locon"
      subprocess.run(["git", "checkout", "04b768bad41e0f1122a86d0636fce47bb2765af8"], cwd=locon_path)
    if opt_value == 'ToMe':
      !pip install tomesd  > /dev/null 2>&1
      subprocess.run(["git", "clone", "--depth", "1", "https://github.com/SLAPaper/a1111-sd-webui-tome"], cwd="/content/stable-diffusion-webui/extensions-builtin")
    os.chdir('/content/stable-diffusion-webui')
    #скрипты
    scrurls = [
        "https://github.com/feffy380/prompt-morph/raw/master/prompt_morph.py",
        "https://github.com/Animator-Anon/Animator/raw/main/animation.py",
        "https://github.com/Animator-Anon/Animator/raw/main/animation_v6.py",
        "https://github.com/memes-forever/Stable-diffusion-webui-video/raw/main/videos.py",
        "https://github.com/dfaker/embedding-to-png-script/raw/main/embedding_to_png.py",
        "https://github.com/dfaker/stable-diffusion-webui-cv2-external-masking-script/raw/main/external_masking.py",
    ]
    def download_sdscripts(scrurl):
      !$dl $scrurl -P /content/stable-diffusion-webui/scripts
    with ThreadPoolExecutor(max_workers=len(scrurls)) as executor:
      futures = [executor.submit(download_sdscripts, scrurl) for scrurl in scrurls]
      for future in as_completed(futures):
          future.result()
    clear()
  except Exception as e:
    try:
      e = "в функции extentions_builtin и получил "+str(e)
      error_message = str(cgitb.text(sys.exc_info()))
      ErrorReport(e, error_message)
      print("Ошибка: ", e)
    except Exception as e:
      e = "в функции extentions_builtin и получил "+str(e)
      import traceback
      error_message = str(traceback.format_exc())
      ErrorReport(e, error_message)
      print("Ошибка: ", error_message)

# скачивание не обязательных дополнений/скриптов
def extentions_install():
  # f"git clone https://github.com/dfaker/SD-latent-mirroring /content/stable-diffusion-webui/extensions/latent-mirroring",
  try:
    if extentions.value == True:
      sdextentions = [
      f"git clone https://github.com/hako-mikan/sd-webui-regional-prompter /content/stable-diffusion-webui/extensions/sd-webui-regional-prompter",
      f"git clone https://github.com/OedoSoldier/enhanced-img2img /content/stable-diffusion-webui/extensions/enhanced-img2img",
      f"git clone https://github.com/wcde/custom-hires-fix-for-automatic1111 /content/stable-diffusion-webui/extensions/custom-hires-fix",
      ]
      def ext_sdextention_inst(sdextention):
        !$sdextention
      with ThreadPoolExecutor(max_workers=len(sdextentions)) as executor:
        futures = [executor.submit(ext_sdextention_inst, sdextention) for sdextention in sdextentions]
        for future in as_completed(futures):
            future.result()
    clear()
  except Exception as e:
    try:
      error_message = str(cgitb.text(sys.exc_info()))
      e = "в функции extentions_install и получил "+str(e)
      ErrorReport(e, error_message)
      print("Ошибка: ", e)
    except Exception as e:
      import traceback
      e = "в функции extentions_install и получил "+str(e)
      error_message = str(traceback.format_exc())
      ErrorReport(e, error_message)
      print("Ошибка: ", error_message)
  if extentions.value == False:
    None

# дополнения для миксов
def extentions_mix_install():
  try:
    if extentions_mix.value == True:
      mixextentions = [
      f"git clone https://github.com/hako-mikan/sd-webui-supermerger /content/stable-diffusion-webui/extensions/sd-webui-supermerger",
      f"git clone https://github.com/arenatemp/stable-diffusion-webui-model-toolkit /content/stable-diffusion-webui/extensions/stable-diffusion-webui-model-toolkit",
      f"git clone https://github.com/bbc-mc/sdweb-merge-block-weighted-gui /content/stable-diffusion-webui/extensions/sdweb-merge-block-weighted-gui",
      f"git clone https://github.com/bbc-mc/sdweb-merge-board /content/stable-diffusion-webui/extensions/sdweb-merge-board",
      ]
      def mixextention_inst(mixextention):
        !$mixextention
      with ThreadPoolExecutor(max_workers=len(mixextentions)) as executor:
        futures = [executor.submit(mixextention_inst, mixextention) for mixextention in mixextentions]
        for future in as_completed(futures):
            future.result()
    clear()
  except Exception as e:
    try:
      e = "в функции extentions_mix_install и получил "+str(e)
      error_message = str(cgitb.text(sys.exc_info()))
      ErrorReport(e, error_message)
      print("Ошибка: ", e)
    except Exception as e:
      e = "в функции extentions_mix_install и получил "+str(e)
      import traceback
      error_message = str(traceback.format_exc())
      ErrorReport(e, error_message)
      print("Ошибка: ", error_message)
  if extentions_mix.value == False:
    None

# симлинки лор в папке дополнения
def lora_to_extension():
  lpath = "/content/stable-diffusion-webui/models/Lora"
  extpath = "/content/stable-diffusion-webui/extensions/sd-webui-additional-networks/models/lora"
  os.makedirs(lpath, exist_ok=True)
  os.makedirs(extpath, exist_ok=True)
  for l in os.listdir(lpath):
    try:
      os.symlink(f"{lpath}/{l}", f"{extpath}/{l}")
    except FileExistsError:
      pass

#скачивание интеррогейтов и языковых моделей для составления промпта (быстрее чем через вебуи)
def misc_models():
  try:
    miscmodels = [
    f"{wget} https://github.com/AUTOMATIC1111/TorchDeepDanbooru/releases/download/v1/model-resnet_custom_v3.pt -P /content/stable-diffusion-webui/models/torch_deepdanbooru",
    f"{wget} https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_base_caption_capfilt_large.pth -P /content/stable-diffusion-webui/models/BLIP",
    f"wget -q {gh}artists.txt -P /content/stable-diffusion-webui/interrogate",
    f"wget -q {gh}flavors.txt -P /content/stable-diffusion-webui/interrogate",
    f"wget -q {gh}mediums.txt -P /content/stable-diffusion-webui/interrogate",
    f"wget -q {gh}movements.txt -P /content/stable-diffusion-webui/interrogate",
    ]
    def misc_models_inst(miscmodel):
      !$miscmodel
    with ThreadPoolExecutor(max_workers=len(miscmodels)) as executor:
      futures = [executor.submit(misc_models_inst, miscmodel) for miscmodel in miscmodels]
      for future in as_completed(futures):
          future.result()
    clear()
  except Exception as e:
    try:
      error_message = str(cgitb.text(sys.exc_info()))
      e = "в функции misc_models и получил "+str(e)
      ErrorReport(e, error_message)
      print("Ошибка: ", e)
    except Exception as e:
      import traceback
      e = "в функции misc_models и получил "+str(e)
      error_message = str(traceback.format_exc())
      ErrorReport(e, error_message)
      print("Ошибка: ", error_message)

# некоторые модели апскелеров
def upscalers():
  try:
    upscmodels = [
    f"wget -q https://huggingface.co/utnah/LDSR/resolve/main/project.yaml -P /content/stable-diffusion-webui/models/LDSR",
    f"wget -nv -t 10 --show-progress --progress=bar -q --content-disposition https://huggingface.co/utnah/LDSR/resolve/main/model.safetensors -P /content/stable-diffusion-webui/models/LDSR",
    f"{dl} {esrgan}4x-UltraSharp.pth -P /content/stable-diffusion-webui/models/ESRGAN",
    f"{dl} {esrgan}4x-AnimeSharp.pth -P /content/stable-diffusion-webui/models/ESRGAN",
    f"{dl} {esrgan}4x_foolhardy_Remacri.pth -P /content/stable-diffusion-webui/models/ESRGAN",
    f"{dl} {esrgan}4xESRGAN.pth -P /content/stable-diffusion-webui/models/ESRGAN",
    f"{dl} {esrgan}4x_Nickelback_70000G.pth -P /content/stable-diffusion-webui/models/ESRGAN",
    f"{dl} {esrgan}lollypop.pth -P /content/stable-diffusion-webui/models/ESRGAN",
    f"{dl} {esrgan}1x_artifacts_jpg_20_40_alsa.pth -P /content/stable-diffusion-webui/models/ESRGAN",
    f"{dl} {esrgan}RealESRGAN_x4plus_anime_6B.pth -P /content/stable-diffusion-webui/models/RealESRGAN",
    f"{dl} {esrgan}16xPSNR.pth -P /content/stable-diffusion-webui/models/ESRGAN",
    f"{dl} {esrgan}4x_NMKD-Siax_200k.pth -P /content/stable-diffusion-webui/models/ESRGAN",
    f"{dl} {esrgan}4x_Valar_v1.pth -P /content/stable-diffusion-webui/models/ESRGAN",
    f"{dl} https://huggingface.co/sberbank-ai/Real-ESRGAN/resolve/main/RealESRGAN_x2.pth -P /content/stable-diffusion-webui/models/RealESRGAN",
    f"{dl} https://huggingface.co/sberbank-ai/Real-ESRGAN/resolve/main/RealESRGAN_x4.pth -P /content/stable-diffusion-webui/models/RealESRGAN",
    f"{dl} {esrgan}4x_NMKD-Siax_200k.pth -P /content/stable-diffusion-webui/models/ESRGAN",
    f"{dl} {esrgan}4x-UltraSharp.pth -P /content/stable-diffusion-webui/models/ESRGAN",
    f"{dl} {esrgan}x1_ITF_SkinDiffDetail_Lite_v1.pth -P /content/stable-diffusion-webui/models/ESRGAN",
    ]
    def upscmodels_inst(upscmodel):
      !$upscmodel
    with ThreadPoolExecutor(max_workers=len(upscmodels)) as executor:
      futures = [executor.submit(upscmodels_inst, upscmodel) for upscmodel in upscmodels]
      for future in as_completed(futures):
          future.result()
    clear()
  except Exception as e:
    try:
      error_message = str(cgitb.text(sys.exc_info()))
      e = "в функции upscalers и получил "+str(e)
      ErrorReport(e, error_message)
      print("Ошибка: ", e)
    except Exception as e:
      import traceback
      e = "в функции upscalers и получил "+str(e)
      error_message = str(traceback.format_exc())
      ErrorReport(e, error_message)
      print("Ошибка: ", error_message)

def emdeddings_install():
  if embeddings.value == True:
    try:
      !$dl $pt'embeddings' -O e.zip
      embunzip = f'7z -bsp1 -bso0 -y -bt x e.zip -o"/content/stable-diffusion-webui/embeddings"'
      !$embunzip
      !rm e.zip
      clear()
    except Exception as e:
      try:
        e = "в функции emdeddings_install и получил "+str(e)
        error_message = str(cgitb.text(sys.exc_info()))
        ErrorReport(e, error_message)
        print("Ошибка: ", e)
      except Exception as e:
        import traceback
        e = "в функции emdeddings_install и получил "+str(e)
        error_message = str(traceback.format_exc())
        ErrorReport(e, error_message)
        print("Ошибка: ", error_message)
  if embeddings.value == False:
    None
def emdeddings_for_negative():
  try:
    !$dl $pt'neg_embeddings' -O n.zip
    nembunzip = f'7z -bsp1 -bso0 -y -bt x n.zip -o"/content/stable-diffusion-webui/embeddings"'
    !$nembunzip
    !rm n.zip
    clear()
  except Exception as e:
    try:
      error_message = str(cgitb.text(sys.exc_info()))
      e = "в функции emdeddings_for_negative и получил "+str(e)
      ErrorReport(e, error_message)
      print("Ошибка: ", e)
    except Exception as e:
      import traceback
      e = "в функции emdeddings_for_negative и получил "+str(e)
      error_message = str(traceback.format_exc())
      ErrorReport(e, error_message)
      print("Ошибка: ", error_message)

# deepdanbooru и автозавершение тегов
def deepdanbooru():
  try:
    if booru_value == True:
      subprocess.run(["git", "clone", "--depth", "1", "https://github.com/DominikDoom/a1111-sd-webui-tagcomplete"], cwd="/content/stable-diffusion-webui/extensions")
      os.makedirs('/content/stable-diffusion-webui/tmp', exist_ok=True)
      tagacp = "/content/stable-diffusion-webui/tmp/tagAutocompletePath.txt"
      !touch $tagacp
      with open(tagacp, 'a') as ta: ta.write("extensions/a1111-sd-webui-tagcomplete/tags")
      !$wget $pt'deepdanbooru-v3-20211112-sgd-e28.zip' -P /content && unzip /content/deepdanbooru-v3-20211112-sgd-e28.zip  -d /content/stable-diffusion-webui/models/deepbooru/ && rm /content/deepdanbooru-v3-20211112-sgd-e28.zip
    if booru_value == False:
      None
  except Exception as e:
    try:
      e = "в функции deepdanbooru и получил "+str(e)
      error_message = str(cgitb.text(sys.exc_info()))
      ErrorReport(e, error_message)
      print("Ошибка: ", e)
    except Exception as e:
      e = "в функции deepdanbooru и получил "+str(e)
      import traceback
      error_message = str(traceback.format_exc())
      ErrorReport(e, error_message)
      print("Ошибка: ", error_message)
  clear()

# скачивание VAE
def VAE():
  try:
    vae_path = "/content/stable-diffusion-webui/models/VAE"
    dl = f"{wget} -P {vae_path} "
    os.makedirs(vae_path, exist_ok=True)
    vaemodels = [
    f"{dl} {vae}blessed-fix.ckpt",
    f"{dl} {vae}blessed.ckpt",
    f"{dl} {vae}blessed2.ckpt",
    f"{dl} {vae}kl-f8-anime.ckpt",
    f"{dl} {vae}kl-f8-anime2.ckpt",
    f"{dl} {vae}kl-f8-trinart_characters.ckpt",
    f"{dl} {vae}novelai-vae.ckpt",
    f"{dl} {vae}vae-ft-ema.ckpt",
    f"{dl} {vae}vae-ft-mse.ckpt",
    f"{dl} {vae}color.vae.safetensors",
    f"{dl} {vae}sdxl_vae.safetensors",
    ]
    def vaemodels_inst(vaemodel):
      !$vaemodel
    with ThreadPoolExecutor(max_workers=len(vaemodels)) as executor:
      futures = [executor.submit(vaemodels_inst, vaemodel) for vaemodel in vaemodels]
      for future in as_completed(futures):
          future.result()
    clear()
  except Exception as e:
    try:
      e = "в функции VAE и получил "+str(e)
      error_message = str(cgitb.text(sys.exc_info()))
      ErrorReport(e, error_message)
      print("Ошибка: ", e)
    except Exception as e:
      e = "в функции VAE и получил "+str(e)
      import traceback
      error_message = str(traceback.format_exc())
      ErrorReport(e, error_message)
      print("Ошибка: ", error_message)

# скачивание гиперсетей
def hypernets_install():
  try:
    if hypernet.value == True:
      hpath = "/content/stable-diffusion-webui/models/hypernetworks"
      dl = f"{wget} -P {hpath} "
      hypernetworks = [
            f"{dl} '{hyper}LuisapAngels_v1.pt'",
            f"{dl} '{hyper}aki99-112000.pt'",
            f"{dl} '{hyper}LuisapOilPainting_v1.pt'",
            f"{dl} '{hyper}anime.pt'",
            f"{dl} '{hyper}anime_2.pt'",
            f"{dl} '{hyper}aini.pt'",
            f"{dl} '{hyper}henriRousseau_v1.pt'",
            f"{dl} '{hyper}gigantomastia_.pt'",
            f"{dl} '{hyper}photo-100k.pt'",
            f"{dl} '{hyper}curssHN slime.pt'",
            f"{dl} '{hyper}nullmono-23000.pt'",
            f"{dl} '{hyper}newhp6.pt'",
            f"{dl} '{hyper}bangyou-hyper-77500.pt'",
            f"{dl} '{hyper}watercolourv1_v10.pt'",
            f"{dl} '{hyper}LuisapMelted_v1.pt'",
            f"{dl} '{hyper}kyokucho-40000.pt'",
            f"{dl} '{hyper}mjv4Hypernetwork_v1.pt'",
            f"{dl} '{hyper}anime_3.pt'",
            f"{dl} '{hyper}molu-60000.pt'",
            f"{dl} '{hyper}spaceMarine_v10.pt'",
            f"{dl} '{hyper}mumumu-hyper-61000.pt'",
            f"{dl} '{hyper}bigrbear-10000.pt'",
            f"{dl} '{hyper}LuisapDarkage_pinguinstyle5Darkage.pt'",
            f"{dl} '{hyper}samdoesartsHypernetwork_v422500Steps.pt'",
            f"{dl} '{hyper}mechanicus_v10.pt'",
            f"{dl} '{hyper}phanTattoo_phantatt.pt'",
            f"{dl} '{hyper}j9063DStyle_.pt'",
          ]
      def hypernetworks_inst(hypernetwork):
        !$hypernetwork
      with ThreadPoolExecutor(max_workers=len(hypernetworks)) as executor:
        futures = [executor.submit(hypernetworks_inst, hypernetwork) for hypernetwork in hypernetworks]
        for future in as_completed(futures):
            future.result()
        clear()
    if hypernet.value == False:
      None
  except Exception as e:
    try:
      error_message = str(cgitb.text(sys.exc_info()))
      e = "в функции hypernets_install и получил "+str(e)
      ErrorReport(e, error_message)
      print("Ошибка: ", e)
    except Exception as e:
      e = "в функции hypernets_install и получил "+str(e)
      import traceback
      error_message = str(traceback.format_exc())
      ErrorReport(e, error_message)
      print("Ошибка: ", error_message)

# скачивание LORA
def LORA():
  try:
    if loras.value == True:
      path = "/content/stable-diffusion-webui/models/Lora/"
      dl = f"{wget} -P {path} "
      os.makedirs(path, exist_ok=True)
      loramodels = [
          f"{dl} {lora}Seiji_Shingen_idol-master.safetensors",
          f"{dl} {lora}PhotoVintage.pt",
          f"{dl} {lora}SilverjowStyle.safetensors",
          f"{dl} {lora}ArtisticJinskyStyle.safetensors",
          f"{dl} {lora}uncutPenis.safetensors",
          f"{dl} {lora}hoacb.safetensors",
          f"{dl} {lora}penguinFrontierStyle.safetensors",
          f"{dl} {lora}anime_penis.safetensors",
          f"{dl} {lora}priapusATarouMentaiko.safetensors",
          f"{dl} {lora}husbando.safetensors",
          f"{dl} {lora}jockstrap.safetensors",
          f"{dl} {lora}aenaluckStyle.safetensors",
          f"{dl} {lora}fConStyle.safetensors",
          f"{dl} {lora}wrestlingSinglets.safetensors",
          f"{dl} {lora}rnarccusStyle.safetensors",
          f"{dl} {lora}unionSuits.safetensors",
          f"{dl} {lora}pissDrinking.safetensors",
          f"{dl} {lora}StepanGilev.safetensors",
          f"{dl} {lora}Lozhkin.safetensors",
          f"{dl} {lora}AntonyStarr_10.safetensors",
          f"{dl} {lora}turbomale_LoraV6.safetensors",
          f"{dl} {lora}gigachadDiffusionLora_v69.safetensors",
          f"{dl} {lora}niceMenModel_menModelV10.safetensors",
          f"{dl} {lora}jacobFryeAssassins_v1.safetensors",
          f"{dl} {lora}sxzLeonSKennedyEduard_sxzLeon.safetensors",
          f"{dl} {lora}kensouTheKingOf_10.safetensors",
          f"{dl} {lora}pantsTugMemeTrend_v1.safetensors",
          f"{dl} {lora}ChiakiMorisawa_v10.safetensors",
          f"{dl} {lora}morphedMuscle_10b.safetensors",
          f"{dl} {lora}husbandoLocon_v31.safetensors",
          f"{dl} {lora}uncutPenisLora_assholeInpanting.safetensors",
          f"{dl} {lora}tekkowangStyle_v1.safetensors",
          f"{dl} {lora}DukeNukem_v1.safetensors",
          f"{dl} {lora}Penis_anime_lora.safetensors",
          f"{dl} {lora}flaccidPenisAnimeLora_v2.safetensors",
          f"{dl} {lora}airoticartsPenisModel_v10Lora.safetensors",
          f"{dl} {lora}onlycocks_v01.safetensors",
          f"{dl} {lora}bettercocks_v10.safetensors",
          f"{dl} {lora}bettercocks_v1Flaccid.safetensors",
        ]
      def dwnld_lora(loramodel):
          !$loramodel
      with ThreadPoolExecutor(max_workers=len(loramodels)) as executor:
          futures = [executor.submit(dwnld_lora, loramodel) for loramodel in loramodels]
          for future in as_completed(futures):
              future.result()
      clear()
    else:
      None
  except Exception as e:
    try:
      error_message = str(cgitb.text(sys.exc_info()))
      e = "в функции LORA и получил "+str(e)
      ErrorReport(e, error_message)
      print("Ошибка: ", e)
    except Exception as e:
      import traceback
      e = "в функции LORA и получил "+str(e)
      error_message = str(traceback.format_exc())
      ErrorReport(e, error_message)
      print("Ошибка: ", error_message)

# дефолтные конфиги
def configs():
  try:
    ng() #нгрок
    defconfigs = [
        f"{dl_gh}localization.json -O /content/stable-diffusion-webui/localizations/localization.json",
        (f"{dl_gh}config_noGD.json -O /content/stable-diffusion-webui/config.json" if checkpoint_value != "newest" else f"{dl_gh}config.json -O /content/stable-diffusion-webui/config.json"),
        (f"{dl} https://raw.githubusercontent.com/PR0LAPSE/StableDiffusionWebUIColab/5911f47ba9cc89835ce280ac8e349a40ffcf4dec/src/ui-config.json -O /content/stable-diffusion-webui/ui-config.json" if checkpoint_value != "newest" else f"{dl_gh}ui-config.json -O /content/stable-diffusion-webui/ui-config.json")
    ]
    def defaultconf(defconfig):
        !$defconfig
    with ThreadPoolExecutor(max_workers=len(defconfigs)) as executor:
        futures = [executor.submit(defaultconf, defconfig) for defconfig in defconfigs]
        for future in as_completed(futures):
            future.result()
    # сохранение картинок на гулодиск
    if gdrive.value == True and os.path.exists("/content/drive/MyDrive"):
      replacer('/content/stable-diffusion-webui/config.json', '/content/output/', '/content/drive/MyDrive/SD/')
      replacer('/content/stable-diffusion-webui/config.json', '"control_net_allow_script_control": true', '"control_net_allow_script_control": true,\n    "def_batch_dir": "/content/drive/MyDrive/SD/ip2p-batch",\n    "outdir_ip2p_samples": "/content/drive/MyDrive/SD/ip2p-images"')
      if os.path.exists("/content/drive/MyDrive/SD/styles.csv"):
        !rm /content/stable-diffusion-webui/styles.csv /content/stable-diffusion-webui/styles.csv.bak
        !ln -s /content/drive/MyDrive/SD/styles.csv /content/stable-diffusion-webui/styles.csv
        !ln -s /content/drive/MyDrive/SD/styles.csv /content/stable-diffusion-webui/styles.csv.bak
        !chmod -c 777 /content/stable-diffusion-webui/styles.csv
        !chmod -c 777 /content/stable-diffusion-webui/styles.csv.bak
      else:
        !rm /content/stable-diffusion-webui/styles.csv /content/stable-diffusion-webui/styles.bak
        !$dl_gh'styles.csv' -O /content/drive/MyDrive/SD/styles.csv
        !ln -s /content/drive/MyDrive/SD/styles.csv /content/stable-diffusion-webui/styles.csv
        !ln -s /content/drive/MyDrive/SD/styles.csv /content/stable-diffusion-webui/styles.csv.bak
        !chmod -c 777 /content/stable-diffusion-webui/styles.csv
        !chmod -c 777 /content/stable-diffusion-webui/styles.csv.bak
    clear()
  except Exception as e:
    try:
      error_message = str(cgitb.text(sys.exc_info()))
      e = "в функции configs и получил "+str(e)
      ErrorReport(e, error_message)
      print("Ошибка: ", e)
    except Exception as e:
      import traceback
      e = "в функции configs и получил "+str(e)
      error_message = str(traceback.format_exc())
      ErrorReport(e, error_message)
      print("Ошибка: ", error_message)
  if gdrive.value == False or (not os.path.exists("/content/drive/MyDrive")):
    None

# англоязычный интерфейс
def eng_lang():
  try:
    if eng.value == True:
      os.remove('/content/stable-diffusion-webui/localizations/localization.json')
      replacer('/content/stable-diffusion-webui/config.json', '"localization": "localization",', '"localization": "",')
    if eng.value == False:
      None
  except Exception as e:
    try:
      error_message = str(cgitb.text(sys.exc_info()))
      e = "в функции eng_lang и получил "+str(e)
      ErrorReport(e, error_message)
      print("Ошибка: ", e)
    except Exception as e:
      e = "в функции eng_lang и получил "+str(e)
      import traceback
      error_message = str(traceback.format_exc())
      ErrorReport(e, error_message)
      print("Ошибка: ", error_message)

# кастомизация
def customisation():
  try:
    !rm /usr/local/lib/python3.10/dist-packages/gradio/strings.py
    !$dl_gh'gradio_strings.py' -O /usr/local/lib/python3.10/dist-packages/gradio/strings.py
    !$dl_gh'notification.mp3' -O /content/stable-diffusion-webui/notification.mp3
    !rm /content/stable-diffusion-webui/html/footer.html /root/.ngrok2/ngrok.yml > /dev/null 2>&1
    !$dl_gh'footer.html' -O /content/stable-diffusion-webui/html/footer.html
    os.environ["DISPLAY"] = "ПРОЛАПС"
    replacer("/content/stable-diffusion-webui/modules/ui.py", "print('ngrok authtoken detected, trying to connect...')", "")
    replacer("/content/stable-diffusion-webui/modules/sd_models.py", 'f"Checkpoint {model_checkpoint} not found; loading fallback {checkpoint_info.title}", file=sys.stderr', '"загрузка модели..."')
    replacer("/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", '                strings.en["RUNNING_LOCALLY_SEPARATED"].format(\n                    self.protocol, self.server_name, self.server_port\n                )', '                  ""')
    replacer("/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", 'strings.en["SHARE_LINK_DISPLAY"].format(self.share_url)', '"\033[01;38;05;112m⯈\033[0m Ссылка на Градио:", self.share_url')
    replacer("/content/stable-diffusion-webui/modules/ngrok.py", "ngrok connected to localhost:{port}! URL: {public_url}\\n'\n               'You can use this link after the launch is complete.", "\\033[01;38;05;112m⯈\\033[0m Ссылка на Нгрок: {public_url}")
    replacer("/content/stable-diffusion-webui/modules/ngrok.py", "f'Invalid ngrok authtoken, ngrok connection aborted.\\n'\n              f'Your token: {token}, get the right one on https://dashboard.ngrok.com/get-started/your-authtoken'", "'указан неверный токен или тебе выпал общий токен который использует другой анон, получи свежий: https://dashboard.ngrok.com/get-started/your-authtoken'")
    replacer('/content/stable-diffusion-webui/javascript/ui.js', 'function restart_reload(){\n    document.body.innerHTML=\'<h1 style="font-family:monospace;margin-top:20%;color:lightgray;text-align:center;">Reloading...</h1>\';\n    setTimeout(function(){location.reload()},2000)\n\n    return []\n}', 'function restart_reload(){\n    document.body.innerHTML=\'<h1 style="font-family:monospace;margin-top:20%;color:red;text-align:center;">Чтобы выполнить перезагрузку, нужно остановить выполнение кода в ячейке колаба:<br>Ctrl+M+I или "Среда выполнения -> Остановить выполнение кода"<br>Затем нужно выполнить перезапуск через ячейку перезапуска!<br>Штатная функция перезапуска ВебУИ не работает через колаб!!!<br>Останавливать и перезапускать нужно каждый раз после добавления/изменения дополнений, списка экстрасетей, скриптов, и т.п.</h1>\';\n    setTimeout(function(){location.reload()},40000)\n\n    return []\n}')
    replacer("/usr/local/lib/python3.10/dist-packages/pyngrok/conf.py", """DEFAULT_NGROK_CONFIG_PATH = os.path.join(os.path.expanduser("~"), ".ngrok2", "ngrok.yml")""", '''DEFAULT_NGROK_CONFIG_PATH = "/root/.config/ngrok/ngrok.yml"''')
    replacer("/content/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/util.py", """if verbose:\n        print(f"{model.__class__.__name__} has {total_params*1.e-6:.2f} M params.")""", "")
    replacer("/content/stable-diffusion-webui/extensions-builtin/LDSR/sd_hijack_ddpm_v1.py", """print(f"{self.__class__.__name__}: Running in {self.parameterization}-prediction mode")""", "")
    replacer("/content/stable-diffusion-webui/modules/models/diffusion/ddpm_edit.py", """print(f"{self.__class__.__name__}: Running in {self.parameterization}-prediction mode")""", "")
    replacer("/content/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/models/diffusion/ddpm.py", """print(f"{self.__class__.__name__}: Running in {self.parameterization}-prediction mode")""", "")
    replacer("/content/stable-diffusion-webui/webui.py", """print(f"Startup time: {startup_timer.summary()}.")""", "")
    replacer("/content/stable-diffusion-webui/webui.py", """print(f"Startup time: {startup_timer.summary()}.")""", "")
    replacer("/content/stable-diffusion-webui/webui.py", """print(f'Interrupted with signal {sig} in {frame}')""", "")
    replacer("/content/stable-diffusion-webui/modules/sd_models.py", """print(f"Model loaded in {timer.summary()}.")""", "")
    replacer("/content/stable-diffusion-webui/extensions/sd-webui-additional-networks/scripts/model_util.py", """print("[AddNet] Updating model hashes...")""", "")
    replacer("/content/stable-diffusion-webui/extensions/sd-webui-additional-networks/scripts/model_util.py", """thread_count = max(1, int(shared.opts.data.get("additional_networks_hash_thread_count", 1)))""", """thread_count = 4""")
  except Exception as e:
    try:
      error_message = str(cgitb.text(sys.exc_info()))
      e = "в функции customisation и получил "+str(e)
      ErrorReport(e, error_message)
      print("Ошибка: ", e)
    except Exception as e:
      import traceback
      e = "в функции customisation и получил "+str(e)
      error_message = str(traceback.format_exc())
      ErrorReport(e, error_message)
      print("Ошибка: ", error_message)
  #clear()

# удалить пустые папки .ipynb_checkpoints и мусор
def cleanup():
  try:
    shutil.rmtree("/content/sample_data")
  except:
    None
  sdwu = "/content/stable-diffusion-webui/"
  junk = ["webui.sh", "webui.bat", "webui-user.sh", "webui-user.bat", "webui-macos-env.sh", "screenshot.png", "README.md", "LICENSE.txt", "CODEOWNERS",]
  dirs_to_remove = glob.glob("/content/.ipynb_checkpoints", recursive=True)
  [os.remove(os.path.join(sdwu, file)) for file in junk if os.path.exists(os.path.join(sdwu, file))]
  for dir in dirs_to_remove:
    path = os.path.join(sdwu, dir)
    try:
      shutil.rmtree(path)
    except:
      None
  if os.path.exists("/content/stable-diffusion-webui/models/Stable-diffusion/nullModel.ckpt"):
    os.remove("/content/stable-diffusion-webui/models/Stable-diffusion/nullModel.ckpt")
  if os.path.exists("/root/.ngrok2/ngrok.yml"):
    try:
      shutil.move("/root/.ngrok2/ngrok.yml", "/root/.config/ngrok/ngrok.yml")
    except:
      None

def complite():
  try:
    !pip install Pillow==8.4.0 > /dev/null 2>&1 # не помню для кого эта версия
    clear()
    import psutil
    from subprocess import getoutput
    from IPython.display import HTML
    total_ram = round(psutil.virtual_memory().total / (1024**3), 2)
    free_ram = round(psutil.virtual_memory().available / (1024**3), 2)
    allocated_ram = round(total_ram - free_ram, 2)
    percent = round((free_ram / total_ram) * 100, 2)
    display(HTML("""<style>.setup_complite{color:var(--ansi-green);font-weight:700}hr.hr_setup_comlite{width:88%;position:relative;right:50%;border:none;border-top:2px var(--ansi-green) solid}.memory{color:var(--ansi-blue);font-weight:bold}.mem_allocate,.mem_all{color:var(--ansi-green)}.percent{color:var(--colab-primary-text-color)}.little_memory{display:flex;flex-wrap:wrap;width:50%;color:var(--ansi-red);font-weight:bold;font-family:sans-serif!important}.cell_fix{font-family:Consolas,monospace!important;color:var(--ansi-blue)}.cell_fix:before{content:"«";font-family:sans-serif!important;color:var(--ansi-blue)}.cell_fix:after{content:"»";font-family:sans-serif!important;color:var(--ansi-blue)}</style>"""))
    display(HTML(f'<p class="setup_complite">Установка завершена<hr class="hr_setup_comlite"/><span class="memory">Доступно ОЗУ: <span class="mem_allocate">{free_ram} Гб</span> из <span class="mem_all">{total_ram} Гб</span> <span class="percent">({percent}% свободно)</span></span></p>'))
    if int(free_ram) <= 7:
      display(HTML('<p class="little_memory">Осталось слишком мало памяти!<br/>Рекомендуется остановить ячейку и выполнить сброс памяти через ячейку <span class="cell_fix">фикс утечки памяти</span>, что находится выше</p>'))
    nsmi = getoutput('nvidia-smi')
    if "nvidia-smi: not found" in nsmi or "NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running." in nsmi:
      display(HTML(f'{gh}warn'))
    else:
      try:
        card = getoutput('nvidia-smi --query-gpu=gpu_name --format=csv').split('\n', 1)[1]
        vram = getoutput('nvidia-smi --query-gpu=memory.total --format=csv').split('\n', 1)[1]
        print("Видеокарта: \033[01;38;05;46mNvidia " + card + "\033[0m |" + "\033[01;38;05;188m Объём видеопамяти: \033[01;38;05;46m" + vram + "\033[0m ")
      except:
        print("nvidia-smi не ответил...")
    aloud("finish.wav")
    if starton_value == False:
      if opt_value == 'SDP' or opt_value == 'ToMe':
        xf = '--opt-sdp-attention'
      if opt_value == 'xformers':
        xf = '--xformers'
      if opt_value == 'nothing':
        xf = '--opt-split-attention'
      if no_half_vae_value == True:
        nhv = '--no-half-vae'
      if no_half_vae_value == False:
        nhv = ''
      if (checkpoint_value != "newest") and (checkpoint_value != "previous"):
        gq = ''
      else:
        gq = '--gradio-queue --disable-console-progressbars --no-hashing --opt-channelslast'
      ckpt_dir = "/content/stable-diffusion-webui/models/Stable-diffusion"
      if booru_value == True:
        deepdanbooru_arg = "--deepdanbooru"
      if booru_value == False:
        deepdanbooru_arg = ""
      default_args = f"--disable-safe-unpickle --enable-insecure-extension-access --share --listen {deepdanbooru_arg} --gradio-img2img-tool editor --theme dark --ckpt-dir {ckpt_dir}"
      ngrok_share = cfg+'cfg'
      freetoken = open(ngrok_share).read().split('\n')
      sharetoken = random.choice(list(freetoken))
      token = ngrok_token
      if not ngrok_token_value or len(ngrok_token_value) < 48 or len(ngrok_token_value) > 51:
        ngrok = f'--ngrok {sharetoken} --ngrok-region eu'
      else:
        ngrok = f'--ngrok {ngrok_token_value} --ngrok-region eu'
      cmdargs = f'{default_args} {gq} {xf} {nhv} {ngrok}'
      # сохранение переменных
      args_data = (xf, nhv, gq, deepdanbooru_arg, default_args)
      with open(f'{cfg}webui_args', 'wb') as webui_args: pickle.dump(args_data, webui_args)
  except Exception as e:
    try:
      error_message = str(cgitb.text(sys.exc_info()))
      e = "в функции complite и получил "+str(e)
      ErrorReport(e, error_message)
      print("Ошибка: ", e)
    except Exception as e:
      import traceback
      e = "в функции complite и получил "+str(e)
      error_message = str(traceback.format_exc())
      ErrorReport(e, error_message)
      print("Ошибка: ", error_message)

def run_webui():
  if starton_value == True:
    try:
      !sleep 3
      aloud("start.mp3")
      !sleep 3
      #Установка токена ngrok
      ngrok_share = cfg+'cfg'
      freetoken = open(ngrok_share).read().split('\n')
      sharetoken = random.choice(list(freetoken))
      token = ngrok_token
      if not ngrok_token_value or len(ngrok_token_value) < 48 or len(ngrok_token_value) > 51:
        ngrok = f'--ngrok {sharetoken} --ngrok-region eu'
      else:
        ngrok = f'--ngrok {ngrok_token_value} --ngrok-region eu'
      if opt_value == 'SDP' or opt_value == 'ToMe':
        xf = '--opt-sdp-attention'
      if opt_value == 'xformers':
        xf = '--xformers'
      if opt_value == 'nothing':
        xf = '--opt-split-attention'
      if no_half_vae_value == True:
        nhv = '--no-half-vae'
      if no_half_vae_value == False:
        nhv = ''
      if (checkpoint_value != "newest") and (checkpoint_value != "previous"):
        gq = ''
      else:
        gq = '--gradio-queue --disable-console-progressbars --no-hashing --opt-channelslast'
      ckpt_dir = "/content/stable-diffusion-webui/models/Stable-diffusion"
      if booru_value == True:
        deepdanbooru_arg = "--deepdanbooru"
      if booru_value == False:
        deepdanbooru_arg = ""
      if os.path.exists("/content/stable-diffusion-webui/extensions/a1111-sd-webui-locon"):
        lyco_arg = ""
      elif os.path.exists("/content/stable-diffusion-webui/extensions/a1111-sd-webui-lycoris"):
        lyco_arg = "--lyco-dir models/Lora --lyco-patch-lora"
      else:
        lyco_arg = ""
      default_args = f"--disable-safe-unpickle --enable-insecure-extension-access --share --listen {deepdanbooru_arg} {lyco_arg} --gradio-img2img-tool editor --theme dark --ckpt-dir {ckpt_dir}"
      cmdargs = f'{default_args} {gq} {xf} {nhv} {ngrok}'
      # сохранение переменных
      args_data = (xf, nhv, gq, deepdanbooru_arg, default_args)
      with open(f'{cfg}webui_args', 'wb') as webui_args: pickle.dump(args_data, webui_args)
      # запуск
      print("\033[1;36mВнимание! При проблемах с интерфейсом, используй другую ссылку, или перезагружай страницу, или выполни перезапуск.\033[0m \n")
      print("\033[1;34mПереходить по ссылкам можно после полной прогрузки, ожидай сообщения.\nСылки могут быть открыты автоматически, если в браузере выданы разрешения на переадресации и открытие окон для этого сайта.\033[0m")
      flara()
      print("")
      #Собственно запуск
      auto_opener()
      # в последней версии автоматик добавил аргументы: --no-hashing, можно использовать вместе с --lowram
      startwu = f"python /content/stable-diffusion-webui/webui.py {cmdargs}"
      !rm /root/.ngrok2/ngrok.yml > /dev/null 2>&1
      !$startwu
    except Exception as e:
      try:
        error_message = str(cgitb.text(sys.exc_info()))
        e = "в функции run_webui и получил "+str(e)
        ErrorReport(e, error_message)
        print("Ошибка: ", e)
      except Exception as e:
        e = "в функции run_webui и получил "+str(e)
        import traceback
        error_message = str(traceback.format_exc())
        ErrorReport(e, error_message)
        print("Ошибка: ", error_message)
    if os.path.exists("/content/drive/MyDrive/SD/styles.csv"):
      !cp -f /content/stable-diffusion-webui/styles.csv /content/drive/MyDrive/SD/styles.csv
    display(HTML('/content/.config/end'))
    if send_arts == True:
      try:
        send_images()
      except:
        None
  else:
    print("\033[01;38;05;211mтобой был отменен автозапуск после установки:\n\033[01;38;05;211mчтобы выполнить запуск вебуи, нужно запустить ячейку «запуск/перезапуск webui».")

# фикс жора ОЗУ
def ram_liberator():
    debs = [
        "wget -q http://launchpadlibrarian.net/367274644/libgoogle-perftools-dev_2.5-2.2ubuntu3_amd64.deb",
        "wget -q https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/google-perftools_2.5-2.2ubuntu3_all.deb",
        "wget -q https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libtcmalloc-minimal4_2.5-2.2ubuntu3_amd64.deb",
        "wget -q https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libgoogle-perftools4_2.5-2.2ubuntu3_amd64.deb",
        "apt install -qq libunwind8-dev",
    ]
    def colab_downgrade(deb):
        subprocess.getoutput(deb)
    with ThreadPoolExecutor(max_workers=len(debs)) as executor:
        futures = [executor.submit(colab_downgrade, deb) for deb in debs]
        for future in futures:
            future.result()
    subprocess.getoutput("dpkg -i *.deb && rm  *.deb")

# ver 19.02.2023
# lambda: on_download_click(None),
# interrogate_clip_install(),
def parallels_tasks():
    tasks = [
        extentions_builtin,
        extentions_install,
        extentions_mix_install,
        upscalers,
        emdeddings_install,
        emdeddings_for_negative,
        VAE,
        hypernets_install,
        deepdanbooru,
        instruct_pix2pix,
        ControlNet_install,
        misc_models,
    ]
    with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor: #временно 16 вместо (max_workers=len(tasks)) чтобы не задыхалось
        # шлем каждый task в отдельный executor
        futures = [executor.submit(run_task, task) for task in tasks]
        for future in concurrent.futures.as_completed(futures):
            # ждем завершения, выплевываем ошибки
            try:
                future.result()
            except Exception as e:
              try:
                error_message = str(cgitb.text(sys.exc_info()))
                e = "в функции parallels_tasks и получил "+str(e)
                ErrorReport(e, error_message)
                print("Ошибка: ", e)
              except Exception as e:
                import traceback
                e = "в функции parallels_tasks и получил "+str(e)
                error_message = str(traceback.format_exc())
                ErrorReport(e, error_message)
                print("Ошибка: ", error_message)

def run_task(task):
    # Новый ThreadPoolExecutor для каждого task
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # запускаем каждый task в новом executor
        executor.submit(task)

# еще один костыль: доустановка зависимостей после установки дополнений
def misc_dep_install():
  try:
    print("завершение установки...")
    os.chdir('/content/stable-diffusion-webui')
    loader(inprogress)
    !python launch.py --share --xformers --ngrok test --no-download-sd-model --exit  > /dev/null 2>&1
    if checkpoint_value == "previous":
      subprocess.run(["pip", "install", "gradio==3.28.3"])
    if checkpoint_value == "newest":
      subprocess.run(["pip", "install", "gradio==3.33.0"])
  except Exception as e:
    try:
      error_message = str(cgitb.text(sys.exc_info()))
      e = "в функции misc_dep_install и получил "+str(e)
      ErrorReport(e, error_message)
      print("Ошибка: ", e)
    except Exception as e:
      import traceback
      e = "в функции misc_dep_install и получил "+str(e)
      error_message = str(traceback.format_exc())
      ErrorReport(e, error_message)
      print("Ошибка: ", error_message)
def depinst():
  if checkpoint_value == "newest":
    depinst_newest()
  elif checkpoint_value == "previous":
    depinst_previous()
  elif checkpoint_value == "legacy":
    None
  elif checkpoint_value == "obsolete":
    None
  else:
    None
# все действия после нажатия на кнопку "продолжить"
#  mobile_player()
#  ram_liberator()
#  ultimate_gdrive_symlink()
#  lora_to_extention()
def apply_button_click(b): # выполнение последрвательно всего выше (паузы  внутри самих def)
  try:
    clear()
    print("жди, установка уже началась...")
    run_nb("store_values")
    depinst()
    A1111_webui_install()
    parallels_tasks()
    on_download_click(None)
    LORA()
    configs()
    eng_lang()
    misc_dep_install()
    customisation()
    cleanup()
    complite()
    run_webui()
  except Exception as e:
    try:
      error_message = str(cgitb.text(sys.exc_info()))
      e = "в функции apply_button_click(b) "+str(e)
      ErrorReport(e, error_message)
      print("Ошибка: ", e)
    except Exception as e:
      import traceback
      error_message = str(traceback.format_exc())
      e = "в функции apply_button_click(b) "+str(e)
      ErrorReport(e, error_message)
      print("Ошибка: ", error_message)
